<a id="TableOfContents"></a>
# Table of Contents:
<li><a href='#imports'>Imports</a></li>
<li><a href="#Q1">Question 1</a></li>
<li><a href="#Q2">Question 2</a></li>
<li><a href="#Q3">Question 3</a></li>
<li><a href="#Q4">Question 4</a></li>
<li><a href="#Q5">Question 5</a></li>
<li><a href="#Additional">Additional Question</a></li>

<a id='imports'></a>
### IMPORTS:
<li><a href='#TableOfContents'>Table of Contents</a></li>

In [1]:
# Basic sheiza
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Stat/Exploration
from scipy import stats

# Modeling
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, export_text, plot_tree
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier

# .py files
import acquire
import prepare
import explore
import evaluation

Create a new notebook, random_forests, and work with titanic data to do the following:

In [2]:
# Acquire titanic dataframe
titanic = prepare.prep_titanic()
titanic.sample()

,survived,pclass,sex,age,fare,embark_town,alone,sex_female,sex_male,embark_town_Cherbourg,embark_town_Queenstown,embark_town_Southampton
454,0,3,male,29.699118,8.05,Southampton,1,0,1,0,0,1


In [3]:
# Identify y_cols
y_titanic = 'survived'

In [4]:
# Identify x_cols
x_titanic = titanic.columns.to_list()
x_titanic = [col for col in x_titanic if col not in ['survived', 'sex', 'embark_town']]
x_titanic

['pclass',
 'age',
 'fare',
 'alone',
 'sex_female',
 'sex_male',
 'embark_town_Cherbourg',
 'embark_town_Queenstown',
 'embark_town_Southampton']

In [5]:
# split data and stratify on survived
train, validate, test = prepare.split(titanic)

What column do you want to stratify onsurvived
train.shape:(498, 12)
validate.shape:(214, 12)
test.shape:(179, 12)


In [6]:
# Verify split shapes
train.shape, validate.shape, test.shape

((498, 12), (214, 12), (179, 12))

In [7]:
models = pd.DataFrame(train.survived)
models

,survived
474,0
370,1
573,1
110,0
167,0
...,...
735,0
163,0
770,0
196,0


<a id='Q1'></a>
### 1. Fit the Random Forest classifier to your training sample and transform (i.e. make predictions on the training sample) setting the random_state accordingly and setting min_samples_leaf = 1 and max_depth = 10.
<li><a href='#TableOfContents'>Table of Contents</a></li>

In [8]:
clf1 = RandomForestClassifier(random_state=43110, min_samples_leaf=1, max_depth=10)
clf1.fit(train[x_titanic], train[y_titanic])
model1_rf = clf1.predict(train[x_titanic])
models['model1_rf'] = model1_rf

<a id='Q2'></a>
### 2. Evaluate your results using the model score, confusion matrix, and classification report.
<li><a href='#TableOfContents'>Table of Contents</a></li>

In [9]:
clf1_score = round(clf1.score(train[x_titanic], train[y_titanic]), 2)
clf1_confusion_matrix = confusion_matrix(models.survived, models.model1_rf, labels=(1, 0))
clf1_class_report = classification_report(models.survived, models.model1_rf)
print(f'Score:\n{clf1_score}\n\nConfusion Matrix:\n{clf1_confusion_matrix}\n\nClassification Report:\n{clf1_class_report}')

Score:
0.96

Confusion Matrix:
[[174  17]
 [  2 305]]

Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.99      0.97       307
           1       0.99      0.91      0.95       191

    accuracy                           0.96       498
   macro avg       0.97      0.95      0.96       498
weighted avg       0.96      0.96      0.96       498



<a id='Q3'></a>
### 3. Print and clearly label the following: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.
<li><a href='#TableOfContents'>Table of Contents</a></li>

In [10]:
# Accuracy
evaluation.accuracy(models, 'survived')

survived: 100.00%

model1_rf: 96.18%

HIGHEST VALUE = model1_rf: 96.18%
LOWEST VALUE = model1_rf: 96.18%


In [11]:
# True Positive Rate
round(clf1_confusion_matrix[0, 0] / clf1_confusion_matrix.sum(), 2)

0.35

In [12]:
# False Positive Rate
round(clf1_confusion_matrix[0, 1] / clf1_confusion_matrix.sum(), 2)

0.03

In [13]:
# True Negative Rate
round(clf1_confusion_matrix[1, 0] / clf1_confusion_matrix.sum(), 2)

0.0

In [14]:
# False Negative Rate
round(clf1_confusion_matrix[1, 1] / clf1_confusion_matrix.sum(), 2)

0.61

In [15]:
# Precision
evaluation.precision(models, 'survived', 1, 0)

survived: 100.00%

model1_rf: 91.10%

HIGHEST VALUE = model1_rf: 91.10%
LOWEST VALUE = model1_rf: 91.10%


In [16]:
# Recall
evaluation.recall(models, 'survived', 1, 0)

survived: 100.00%

model1_rf: 98.86%

HIGHEST VALUE = model1_rf: 98.86%
LOWEST VALUE = model1_rf: 98.86%


In [17]:
# F1 Score
evaluation.f1_score(models, 'survived', 1, 0)

survived: 100.00%

model1_rf: 94.82%

HIGHEST VALUE = model1_rf: 94.82%
LOWEST VALUE = model1_rf: 94.82%


In [18]:
# Support
classification_report(models.survived, models.model1_rf)

'              precision    recall  f1-score   support\n\n           0       0.95      0.99      0.97       307\n           1       0.99      0.91      0.95       191\n\n    accuracy                           0.96       498\n   macro avg       0.97      0.95      0.96       498\nweighted avg       0.96      0.96      0.96       498\n'

<a id='Q4'></a>
### 4. Run through steps increasing your min_samples_leaf and decreasing your max_depth.
<li><a href='#TableOfContents'>Table of Contents</a></li>

In [19]:
# min_samples_leaf = 2, max_depth = 9
clf2 = RandomForestClassifier(random_state=43110, min_samples_leaf=2, max_depth=9)
clf2.fit(train[x_titanic], train[y_titanic])
model2_rf = clf2.predict(train[x_titanic])
models['model2_rf'] = model2_rf

In [20]:
# min_samples_leaf = 3, max_depth = 8
clf3 = RandomForestClassifier(random_state=43110, min_samples_leaf=3, max_depth=8)
clf3.fit(train[x_titanic], train[y_titanic])
model3_rf = clf3.predict(train[x_titanic])
models['model3_rf'] = model3_rf

In [21]:
# min_samples_leaf = 4, max_depth = 7
clf4 = RandomForestClassifier(random_state=43110, min_samples_leaf=4, max_depth=7)
clf4.fit(train[x_titanic], train[y_titanic])
model4_rf = clf4.predict(train[x_titanic])
models['model4_rf'] = model4_rf

In [22]:
# min_samples_leaf = 5, max_depth = 6
clf5 = RandomForestClassifier(random_state=43110, min_samples_leaf=5, max_depth=6)
clf5.fit(train[x_titanic], train[y_titanic])
model5_rf = clf5.predict(train[x_titanic])
models['model5_rf'] = model5_rf

In [23]:
# min_samples_leaf = 6, max_depth = 5
clf6 = RandomForestClassifier(random_state=43110, min_samples_leaf=6, max_depth=5)
clf6.fit(train[x_titanic], train[y_titanic])
model6_rf = clf6.predict(train[x_titanic])
models['model6_rf'] = model6_rf

In [24]:
# min_samples_leaf = 7, max_depth = 4
clf7 = RandomForestClassifier(random_state=43110, min_samples_leaf=7, max_depth=4)
clf7.fit(train[x_titanic], train[y_titanic])
model7_rf = clf7.predict(train[x_titanic])
models['model7_rf'] = model7_rf

In [25]:
# min_samples_leaf = 8, max_depth = 3
clf8 = RandomForestClassifier(random_state=43110, min_samples_leaf=8, max_depth=3)
clf8.fit(train[x_titanic], train[y_titanic])
model8_rf = clf8.predict(train[x_titanic])
models['model8_rf'] = model8_rf

In [26]:
# min_samples_leaf = 9, max_depth = 2
clf9 = RandomForestClassifier(random_state=43110, min_samples_leaf=9, max_depth=2)
clf9.fit(train[x_titanic], train[y_titanic])
model9_rf = clf9.predict(train[x_titanic])
models['model9_rf'] = model9_rf

In [27]:
# min_samples_leaf = 10, max_depth = 1
clf10 = RandomForestClassifier(random_state=43110, min_samples_leaf=10, max_depth=1)
clf10.fit(train[x_titanic], train[y_titanic])
model10_rf = clf10.predict(train[x_titanic])
models['model10_rf'] = model10_rf

In [28]:
models.sample()

,survived,model1_rf,model2_rf,model3_rf,model4_rf,model5_rf,model6_rf,model7_rf,model8_rf,model9_rf,model10_rf
294,0,0,0,0,0,0,0,0,0,0,0


<a id='Q5'></a>
### 5. What are the differences in the evaluation metrics? Which performs better on your in-sample data? Why?
<li><a href='#TableOfContents'>Table of Contents</a></li>

In [29]:
evaluation.f1_score(models, 'survived', 1, 0)

survived: 100.00%

model1_rf: 94.82%

model2_rf: 87.29%

model3_rf: 84.18%

model4_rf: 82.72%

model5_rf: 80.81%

model6_rf: 75.53%

model7_rf: 74.55%

model8_rf: 72.52%

model9_rf: 71.78%

model10_rf: 71.78%

HIGHEST VALUE = model1_rf: 94.82%
LOWEST VALUE = model9_rf: 71.78%


Model-1 performed the best via F-1 Score being the highest...

Something to note, as min_samples_leaf increased and max_depth decreased, the scores consistently  got lower

<a id='Additional'></a>
### Additional. After making a few models, which one has the best performance (or closest metrics) on both train and validate?
<li><a href='#TableOfContents'>Table of Contents</a></li>

In [30]:
models_list = [
    clf1, 
    clf2, 
    clf3, 
    clf4, 
    clf5, 
    clf6, 
    clf7, 
    clf8, 
    clf9, 
    clf10
]

In [36]:
train_dict = {}
validate_dict = {}
performance_dict = {}
modelnum = 0
for x in models_list:
    modelnum += 1
    train_score = round(x.score(train[x_titanic], train[y_titanic]), 2)
    validate_score = round(x.score(validate[x_titanic], validate[y_titanic]), 2)
    train_dict[modelnum] = train_score
    validate_dict[modelnum] = validate_score
    diff = abs(train_score - validate_score)
    performance_dict[modelnum] = diff
    print(f'\033[32mmodel{modelnum}\033[0m Train Score: {train_score}')
    print(f'\033[32mmodel{modelnum}\033[0m Validate Score: {validate_score}\n')
train_max_model = max(train_dict, key=train_dict.get)
train_max_pct = train_dict[train_max_model]
train_min_model = min(train_dict, key=train_dict.get)
train_min_pct = train_dict[train_min_model]
validate_max_model = max(validate_dict, key=validate_dict.get)
validate_max_pct = validate_dict[validate_max_model]
validate_min_model = min(validate_dict, key=validate_dict.get)
validate_min_pct = validate_dict[validate_min_model]
performance_lowest_model = max(performance_dict, key=performance_dict.get)
performance_lowest_pct = performance_dict[performance_lowest_model]
performance_highest_model = min(performance_dict, key=performance_dict.get)
performance_highest_pct = performance_dict[performance_highest_model]
print(f'\033[31mHIGHEST VALUE (TRAIN)\033[0m = \033[32mmodel{train_max_model}\033[0m: {train_max_pct}')
print(f'\033[31mLOWEST VALUE (TRAIN)\033[0m = \033[32mmodel{train_min_model}\033[0m: {train_min_pct}')
print(f'\033[31mHIGHEST VALUE (VALIDATE)\033[0m = \033[32mmodel{validate_max_model}\033[0m: {validate_max_pct}')
print(f'\033[31mLOWEST VALUE (VALIDATE)\033[0m = \033[32mmodel{validate_min_model}\033[0m: {validate_min_pct}')
print(f'\033[31mHIGHEST DIFF\033[0m = \033[32mmodel{performance_lowest_model}\033[0m: {performance_lowest_pct}')
print(f'\033[31mLOWEST DIFF\033[0m = \033[32mmodel{performance_highest_model}\033[0m: {performance_highest_pct}')

model1 Train Score: 0.96
model1 Validate Score: 0.78

model2 Train Score: 0.91
model2 Validate Score: 0.78

model3 Train Score: 0.89
model3 Validate Score: 0.79

model4 Train Score: 0.88
model4 Validate Score: 0.79

model5 Train Score: 0.87
model5 Validate Score: 0.79

model6 Train Score: 0.84
model6 Validate Score: 0.78

model7 Train Score: 0.83
model7 Validate Score: 0.79

model8 Train Score: 0.81
model8 Validate Score: 0.75

model9 Train Score: 0.79
model9 Validate Score: 0.74

model10 Train Score: 0.79
model10 Validate Score: 0.74

HIGHEST VALUE (TRAIN) = model1: 0.96
LOWEST VALUE (TRAIN) = model9: 0.79
HIGHEST VALUE (VALIDATE) = model3: 0.79
LOWEST VALUE (VALIDATE) = model9: 0.74
HIGHEST DIFF = model1: 0.17999999999999994
LOWEST DIFF = model7: 0.039999999999999925


Model-7 has the lowest difference between train and validate scores

Model-7 restrictions: 
- min_samples_leaf = 7
- max_depth = 4

In [32]:
# Testing evaluation.py function
evaluation.train_val_scores(train, validate, x_titanic, y_titanic, models_list)

model1 Train Score: 0.96
model1 Validate Score: 0.78

model2 Train Score: 0.91
model2 Validate Score: 0.78

model3 Train Score: 0.89
model3 Validate Score: 0.79

model4 Train Score: 0.88
model4 Validate Score: 0.79

model5 Train Score: 0.87
model5 Validate Score: 0.79

model6 Train Score: 0.84
model6 Validate Score: 0.78

model7 Train Score: 0.83
model7 Validate Score: 0.79

model8 Train Score: 0.81
model8 Validate Score: 0.75

model9 Train Score: 0.79
model9 Validate Score: 0.74

model10 Train Score: 0.79
model10 Validate Score: 0.74

HIGHEST VALUE (TRAIN) = model1: 0.96
LOWEST VALUE (TRAIN) = model9: 0.79
HIGHEST VALUE (VALIDATE) = model3: 0.79
LOWEST VALUE (VALIDATE) = model9: 0.74
HIGHEST DIFF = model1: 0.17999999999999994
LOWEST DIFF = model7: 0.039999999999999925
